In [1]:
!pip install kaggle


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ningom","key":"53b8540e06ccbede8759befaff246b35"}'}

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!mv kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [7]:
!kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 25.4MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 53.6MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 12.0MB/s]


In [0]:
#データ取得
import pandas as pd
import numpy as np
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [56]:
sep_forward = train_df["Name"].str.split(",",expand=True)
sep_backward = sep_forward[1].str.split(".",expand=True)
job = sep_backward[0].str.replace(" ","")
train_df["Title"] = job

display(train_df["Title"].unique())

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'theCountess',
       'Jonkheer'], dtype=object)

In [57]:
display(train_df.groupby(by=["Title"]).count())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Title,,,,,,,,,,,,
Capt,1,1,1,1,1,1,1,1,1,1,1,1
Col,2,2,2,2,2,2,2,2,2,2,1,2
Don,1,1,1,1,1,1,1,1,1,1,0,1
Dr,7,7,7,7,7,6,7,7,7,7,3,7
Jonkheer,1,1,1,1,1,1,1,1,1,1,0,1
Lady,1,1,1,1,1,1,1,1,1,1,1,1
Major,2,2,2,2,2,2,2,2,2,2,2,2
Master,40,40,40,40,40,36,40,40,40,40,7,40
Miss,182,182,182,182,182,146,182,182,182,182,47,181


In [58]:
display(train_df.groupby(by=["Title"])["Survived"].agg(["mean","count"]))

,mean,count
Title,,
Capt,0.000000,1
Col,0.500000,2
Don,0.000000,1
Dr,0.428571,7
Jonkheer,0.000000,1
Lady,1.000000,1
Major,0.500000,2
Master,0.575000,40
Miss,0.697802,182


In [59]:
sep_forward = test_df["Name"].str.split(",",expand=True)
sep_backward = sep_forward[1].str.split(".",expand=True)
job = sep_backward[0].str.replace(" ","")
test_df["Title"] = job
display(test_df.groupby(by=["Title"]).agg(["count"]))

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,count,count,count,count,count,count,count,count,count,count,count
Title,,,,,,,,,,,
Col,2,2,2,2,2,2,2,2,2,2,2
Dona,1,1,1,1,1,1,1,1,1,1,1
Dr,1,1,1,1,1,1,1,1,1,1,1
Master,21,21,21,21,17,21,21,21,21,2,21
Miss,78,78,78,78,64,78,78,78,78,11,78
Mr,240,240,240,240,183,240,240,240,239,42,240
Mrs,72,72,72,72,62,72,72,72,72,32,72
Ms,1,1,1,1,0,1,1,1,1,0,1


In [0]:
from sklearn.preprocessing import LabelEncoder
def data_shape(df):
    df.drop('PassengerId', axis=1, inplace=True)
    df.drop('Ticket', axis=1, inplace=True)
    
    for dataset in df:
        df['Title'] =df['Title'].replace('Master', 'Child')
        df['Title'] =df['Title'].replace(['Mlle','Jonkheer', 'Ms'], 'Miss')
        df['Title'] =df['Title'].replace(['Capt','Col','Don','Dr','Rev','Sir','Major'], 'Mr')
        df['Title'] =df['Title'].replace(['Mme','Lady', 'theCountess','Dona'],'Mrs')
    
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 3}
    df['Title'] = df['Title'].map(title_mapping)
    df['Title'] = df['Title'].fillna(0)
    
    df.drop('Name', axis=1, inplace=True)
    
    df.Embarked = df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
    
    df.Sex = df.Sex.replace(['male', 'female'], [0, 1])
    df.Cabin = df.Cabin.fillna('Unknown')
    df['FamilySize'] = 1+ df['Parch'] + df['SibSp'] 
    for dataset in df:
        df['IsAlone'] = 0
        df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
    # Fareをカテゴリ分け
    df['Fare'].fillna(df['Fare'].median(), inplace = True)
    df['FareCategory'] = pd.qcut(df['Fare'], 4)
    df['FareCategory'] = LabelEncoder().fit_transform(df['FareCategory'])
    df.drop('Fare', axis=1, inplace=True)

    # Ageの欠損を中央値で補間
    
    df['Age'].fillna(df['Age'].median(),inplace=True)
    df.loc[ df['Age'] <= 16, 'Age']= 0
    df.loc[(df['Age'] > 16) & (df['Age'] <= 32), 'Age'] = 1
    df.loc[(df['Age'] > 32) & (df['Age'] <= 48), 'Age'] = 2
    df.loc[(df['Age'] > 48) & (df['Age'] <= 64), 'Age'] = 3
    df.loc[ df['Age'] > 64, 'Age'] = 4 ;
    
    df.drop('SibSp', axis=1, inplace=True)
    df.drop('Parch', axis=1, inplace=True)
    
    levels = []
    # 1文字目を取り出す
    for level in df.Cabin:
        levels.append(level[0])    
    df.Cabin = pd.DataFrame(levels)
    df.Cabin = df.Cabin.replace('T','U')
    df.Cabin = df.Cabin.replace(['A','B','C','D','E','F','G','U'], [0, 1, 2, 3, 4, 5, 6, 7])
    return df

In [61]:
train_df=data_shape(train_df)
test_df=data_shape(test_df)
train_df.head()

,Survived,Pclass,Sex,Age,Cabin,Embarked,Title,FamilySize,IsAlone,FareCategory
0,0,3,0,1.0,7,1.0,1.0,2,0,0
1,1,1,1,2.0,2,0.0,3.0,2,0,3
2,1,3,1,1.0,7,1.0,2.0,1,1,1
3,1,1,1,2.0,2,1.0,3.0,2,0,3
4,0,3,0,2.0,7,1.0,1.0,1,1,1


In [62]:
train_df.isnull().sum()

Survived        0
Pclass          0
Sex             0
Age             0
Cabin           0
Embarked        0
Title           0
FamilySize      0
IsAlone         0
FareCategory    0
dtype: int64

In [0]:
#データの分割(t_train:正解ラベル,x_train:トレーニングデータ)
t_train=train_df["Survived"].values
x_train=train_df
x_train.drop("Survived",axis=1,inplace=True)
x_train=x_train.values

In [0]:
#データの分割
x_test=test_df.values

In [0]:
#正規化
from sklearn.preprocessing import StandardScaler,MinMaxScaler
std_scl = StandardScaler() #MinMaxScaler() or Robustscaler
x_train=std_scl.fit_transform(x_train).astype(np.int64)
x_test=std_scl.fit_transform(x_test).astype(np.int64)

In [0]:
from sklearn.model_selection import train_test_split
#データ分割
X_train,X_valid,Y_train,Y_valid=train_test_split(x_train,t_train,random_state=0)

In [86]:
#モデルの構築(重回帰分析)
#ロジ
from sklearn import linear_model
c = 1.0
model_logistic=linear_model.LogisticRegression(C=c, solver='liblinear', random_state=0)
model_logistic.fit(X_train, Y_train) 

#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
model_clf = RandomForestClassifier(random_state=0)
model_clf.fit(X_train,Y_train)

#サポベク
from sklearn.svm import SVC
model_svc=SVC()
model_svc.fit(X_train,Y_train)

#NN
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,BatchNormalization,Flatten
from keras.optimizers import Adam

batch_size=50
epochs=25

model_nn = Sequential([                       
    Dense(100,input_dim=train_df.shape[1], activation='relu'),
    BatchNormalization(),
    Dense(50, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(25, activation='relu'),
    BatchNormalization(),
    Dense(12, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_nn.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

history=model_nn.fit(X_train, Y_train, epochs=epochs,batch_size=batch_size)

# gradient boost
from sklearn.ensemble import GradientBoostingClassifier 
model_gb=GradientBoostingClassifier(random_state=1)
model_gb.fit(X_train,Y_train)

# XGboost
!pip install xgboost
from xgboost import XGBClassifier
model_xgb = XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=100)
model_xgb.fit(X_train,Y_train)


Epoch 1/25
668/668 [==============================] - 2s 3ms/step - loss: 0.9698 - acc: 0.5314
Epoch 2/25
668/668 [==============================] - 0s 211us/step - loss: 0.7907 - acc: 0.6332
Epoch 3/25
668/668 [==============================] - 0s 215us/step - loss: 0.6761 - acc: 0.6542
Epoch 4/25
668/668 [==============================] - 0s 220us/step - loss: 0.6111 - acc: 0.7186
Epoch 5/25
668/668 [==============================] - 0s 233us/step - loss: 0.5764 - acc: 0.7365
Epoch 6/25
668/668 [==============================] - 0s 233us/step - loss: 0.5836 - acc: 0.7560
Epoch 7/25
668/668 [==============================] - 0s 208us/step - loss: 0.5520 - acc: 0.7530
Epoch 8/25
668/668 [==============================] - 0s 221us/step - loss: 0.5553 - acc: 0.7575
Epoch 9/25
668/668 [==============================] - 0s 212us/step - loss: 0.5190 - acc: 0.7859
Epoch 10/25
668/668 [==============================] - 0s 218us/step - loss: 0.5492 - acc: 0.7665
Epoch 11/25
668/668 [==========

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [87]:
from sklearn import metrics
predicted_logistic = model_logistic.predict(x_train)
predicted_clf = model_clf.predict(x_train)
predicted_svc = model_svc.predict(x_train)
train_loss, train_acc = model_nn.evaluate(x_train, t_train)
predicted_gb = model_gb.predict(x_train)
predicted_xgb = model_xgb.predict(x_train)
expected = t_train
print("Model Accuracy")
print("Logistic:%f"% metrics.accuracy_score(expected,predicted_logistic))
print("Random  :%f"% metrics.accuracy_score(expected,predicted_clf))
print("SVC     :%f"% metrics.accuracy_score(expected,predicted_svc))
print("NN      :%f"% train_acc )
print("Gradient:%f"% metrics.accuracy_score(expected,predicted_gb))
print("XGboost :%f"% metrics.accuracy_score(expected,predicted_xgb))

891/891 [==============================] - 1s 730us/step
Model Accuracy
Logistic:0.795735
Random  :0.850730
SVC     :0.826038
NN      :0.830527
Gradient:0.830527
XGboost :0.836139


In [0]:
# base predicts
base_pred_1 = model_logistic.predict(X_train)
base_pred_2 = model_clf.predict(X_train)
base_pred_3 = model_svc.predict(X_train)
base_pred_4 = np.round(model_nn.predict(X_train))
base_pred_5 = model_gb.predict(X_train)
base_pred_6 = model_xgb.predict(X_train)

In [0]:
# stack base predicts for training meta model
stacked_predictions = np.column_stack((base_pred_1, base_pred_2, base_pred_3,base_pred_4,base_pred_5,base_pred_6))

In [90]:
# train meta model 
meta_model = linear_model.LinearRegression()
meta_model.fit(stacked_predictions, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# final result 
valid_pred_1 = model_logistic.predict(X_valid)
valid_pred_2 = model_clf.predict(X_valid)
valid_pred_3 = model_svc.predict(X_valid)
valid_pred_4 = np.round(model_nn.predict(X_valid))
valid_pred_5 = model_gb.predict(X_valid)
valid_pred_6 = model_xgb.predict(X_valid)
stacked_valid_predictions = np.column_stack((valid_pred_1, valid_pred_2, valid_pred_3,valid_pred_4,valid_pred_5,valid_pred_6))

meta_valid_pred = np.round(meta_model.predict(stacked_valid_predictions))



In [92]:
#モデル評価
from sklearn.metrics import accuracy_score
acc_train=accuracy_score(meta_valid_pred,Y_valid)
print(acc_train)

0.8161434977578476


In [0]:
#予測
test_pred_1 = model_logistic.predict(x_test)
test_pred_2 = model_clf.predict(x_test)
test_pred_3 = model_svc.predict(x_test)
test_pred_4 = np.round(model_nn.predict(x_test))
test_pred_5 = model_gb.predict(x_test)
test_pred_6 = model_xgb.predict(x_test)

stacked_test_predictions = np.column_stack((test_pred_1, test_pred_2, test_pred_3,test_pred_4,test_pred_5,test_pred_6))

y_test=np.round(meta_model.predict(stacked_test_predictions))

submit_df=pd.DataFrame(passengerId_data.T,columns=["PassengerId"])
submit_df["Survived"]=y_test.T

In [0]:
#csvへ
submit_df_int=submit_df.astype(np.int64)
submit_df_int.to_csv("result.csv",index=False)

In [76]:
submit_df_int

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [77]:
!kaggle competitions submit -c titanic -f result.csv -m 'second submit'

100% 2.77k/2.77k [00:00<00:00, 13.2kB/s]
Successfully submitted to Titanic: Machine Learning from Disaster